In [23]:
#*** INITIALISATION ***#

import os
from pathlib import Path
from dotenv import load_dotenv
import datetime

load_dotenv()


# Création d'un dossier temporaire vide

tmp_dir: Path=Path(os.getenv('TMP_DIR'))

if os.path.isdir(tmp_dir):
    os.removedirs(tmp_dir)

os.mkdir(tmp_dir)


# Timestamp
date_now = datetime.date.today().isoformat()
datetime_now = datetime.datetime.now().isoformat()

In [24]:
#*** TÉLÉCHARGEMENT ***#

import pandas as pd
from requests import get

# decp_augmente_valides_file: Path = Path(f'data/decp_augmente_valides_{date_now}.csv')
decp_augmente_valides_file: Path = Path(f'data/decp_augmente_valides_2024-04-12.csv')


if not(os.path.exists(decp_augmente_valides_file)):
    request = get(os.getenv('DECP_ENRICHIES_VALIDES_URL'))
    with open(decp_augmente_valides_file, 'wb') as file:
        file.write(request.content)
else:
    print("DECP d'aujourd'hui déjà téléchargées")

df: pd.DataFrame = pd.read_csv(decp_augmente_valides_file, sep=";", dtype='object')   


DECP d'aujourd'hui déjà téléchargées


In [25]:
#*** ANALYSE DE BASE ***#

# df.info(verbose=True)
# obsolete number of rows 994123
# valid number of rows 837115

In [26]:
# def new_func():
#     for tech in  df['technique'].unique():
#         print(tech)

from numpy import NaN

df.replace([NaN, None], "", inplace=True, regex=False)
# df[['id', 'datePublicationDonnees']].loc[df['datePublicationDonnees'].str.contains("September")]

In [37]:
import numpy

#*** REDRESSEMENT ***#

df.replace([numpy.NaN, None], "", inplace=True, regex=False)

# Dates

columns_date = ["datePublicationDonnees", "dateNotification"]

date_replacements = {
    # ID marché invalide et SIRET de l'acheteur
    "0002-11-30": "",
    "September, 16 2021 00:00:00": "2021-09-16", #20007695800012 19830766200017 (plein !)
    "16 2021 00:00:00": "",
    "0222-04-29": "2022-04-29", # 202201L0100
    "0021-12-05": "2022-12-05", # 20222022/1400
    "0001-06-21": "", #0000000000000000 21850109600018
    "0019-10-18": "", #0000000000000000 34857909500012
    "5021-02-18": "2021-02-18", #20213051200 21590015000016   
    "2921-11-19": "", # 20220057201 20005226400013
    "0022-04-29": "2022-04-29", # 2022AOO-GASL0100 25640454200035
    
    # "/": "-"
}

for col in columns_date:
    df[col] = df[col].replace(date_replacements, regex=False)

# Identifiants de marchés

id_replacements = {
    "[,\./]": "_"
}

df["id"] = df["id"].replace(id_replacements, regex=True)




In [36]:

# df["datePublicationDonnees"].loc[df["datePublicationDonnees"] == '2921-11-19']
df[['id', 'acheteur.id', 'dateNotification', 'datePublicationDonnees']].loc[df['datePublicationDonnees'].str.contains("0022-04-29")]
# df.loc[2067]

,id,acheteur.id,dateNotification,datePublicationDonnees
476337,2022AOO-GASL0100,25640454200035,2022-08-05,0022-04-29


In [34]:
#***  TYPES DE DONNÉES ***#

numeric_dtypes = ["dureeMois", "montant"]

for column in numeric_dtypes:
    df[column] = pd.to_numeric(df[column])


date_dtypes = ["datePublicationDonnees", "dateNotification"]

for column in date_dtypes:
    df[column] = pd.to_datetime(df[column], format='mixed', dayfirst=True)


df.info(verbose=True)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 0022-04-29, at position 2213

In [ ]:
#***  ANALYSES AVANCÉES ***#

# Les sources d'où proviennent les données
sources = df["source"].unique()

for source in sources:
    print(f"""
# {source}
Nombre de marchés : {df[["source"]].loc[df["source"]==source].index.size}
Nombre d'acheteurs uniques : {len(df[["idAcheteur", "source"]].loc[df["source"]==source]['idAcheteur'].unique())}

          """)

In [ ]:
titulaires = pd.concat(df[''])


AWS-Achat 6662
API AIFE 3563
PES Marchés 8841
Dematis 1561
Territoires numériques BFC 403
Megalis Bretagne 713
None 0
Grand Lyon 1


16546

In [ ]:
# VERS LE FORMAT JSON OFFICIEL #